In this code snippet, we are importing several libraries that will be used in our ETF project:

- ***yfinance*** is a popular library for retrieving financial data from Yahoo Finance.
- ***matplotlib*** and ***seaborn*** are plotting libraries that will allow us to create visualizations of our data.
- ***pandas*** is a library for data manipulation and analysis.
- ***numpy*** is a library for numerical computing.

In [ ]:
# Selecting libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

Next, we define a list of stocks that we will be tracking in our ETF project. In this case, we have chosen to track four assets: 
1. ***SPY*** (an ETF that tracks the S&P 500)
2. ***GLD*** (an ETF that tracks the price of gold)
3. ***^TNX*** (the yield of the 10-year U.S. Treasury note)
4. ***^IRX*** (the yield of the 13-week U.S. Treasury bill).

We also define some initial values for our project:

- ***initial_value*** is the amount of money we will be investing in our ETF project. In this case, we have set it to $1,000,000.
- ***initial_etf_shares*** is the number of shares of our ETF that we will initially distributed. In this case, we have set it to 100,000.
- ***allocation*** is the proportion of our initial investment that we will allocate to each asset in our portfolio. In this case, we have chosen to allocate 25% to each asset.
- ***start_date*** is the date to start the data retrieval from.
- The target for the example is to mantain a 25% distribution per asset rebalancing daily. This can be done monthly, quarterly, yearly, etc. The tool will be rebalance the shares weights.

In [ ]:
# Define the assets and key initial metrics as investment amount to generate a portfolio and the number of ETF shares created
stocks = ['SPY', 'GLD', '^TNX', '^IRX']
initial_value = 10**6 
initial_etf_shares = 10**5
allocation = 0.25
start_date = '2005-01-01'

On this simple step, we will retreive the data from ***yfinance*** library. We will clean the data since few missing values exsists and we will store the prices ***values*** for later usage.
We will also calculate the log_returns for the assets and clean the first NaN value.

In [ ]:
# Download data, clean and isolate values for calculation
stock_data = yf.download(stocks, start=start_date)['Adj Close']
stock_data = stock_data.dropna()
stock_prices = stock_data[stocks].values

In [ ]:
for asset in stock_data:
    stock_data[f'{asset}_ret'] =  np.log(stock_data[f'{asset}'] / stock_data[f'{asset}'].shift(1))
stock_data= stock_data.dropna()

In [ ]:
stock_data.head()

The following code calculates the ***daily rebalance*** of a portfolio our four assets.

First, a DataFrame is created to store the number of shares of each asset in the portfolio. The initial number of shares for each asset is calculated based on the initial allocation, which is 25% for each asset, and the initial value of the portfolio, which is 1 million USD.

Then:
- The code iterates over each day in the DataFrame containing the daily prices of the assets and updates the number of shares to maintain the allocation. 
- For each day, the current value of the portfolio is calculated by multiplying the number of shares of each asset by its corresponding price. 
- The target value for each asset is the product of the current value of the portfolio and the allocation for that asset.
- The target number of shares is calculated by dividing the target value by the price of the asset for that day. 
- The number of shares for each asset is then updated in the shares DataFrame.

Finally, the code rounds down the number of shares to the nearest integer, as it is not possible to buy fractional shares. The resulting DataFrame contains the daily number of shares for each asset. ***If you are looking for other asset such cryptocurrencies, fractional shares, etc, you shouldn't floor the result.***

In [ ]:
### Daily rebalance process ### if you want to do monthly, quarterly or yearly this is your area to to work.
 
# Calculate the initial number of shares for each asset based on the initial allocation
shares_df = pd.DataFrame(index=[stock_data.index[0]])
for asset in stocks:
    initial_shares = (initial_value * allocation) / stock_data[asset][stock_data.index[0]]
    shares_df[asset + '_shares'] = initial_shares

# Iterate over each day in the year and update the number of shares to maintain the allocation
for day in stock_data.index:
    if day == stock_data.index[0]:
        continue
    # Calculate the current value of the portfolio
    current_value = 0
    for asset in stocks:
        current_value += shares_df[asset + '_shares'][stock_data.index[0]] * stock_data[asset][day]
    # Update the number of shares for each asset
    for asset in stocks:
        target_value = current_value * allocation
        target_shares = target_value / stock_data[asset][day]
        shares_df.loc[day, asset + '_shares'] = target_shares

# Rounded since no fractional shares/positions
for i in shares_df.columns:
    shares_df[i] = np.floor(shares_df[i])  

shares_df.head()

Then calculates the value for each asset in the portfolio $value = price * shares $, and determines how many shares of each asset need to be bought or sold to bring the portfolio back to the desired allocation on the ***trade_{asset}_shares***. This will order to place. 

- First calculates the value and shares for each asset by multiplying the asset's price by the number of shares held, and stores this information in new columns in the stock_data DataFrame.
- Second calculates the amount of shares that need to be traded to maintain the desired allocation by taking the difference between the current number of shares and the previous number of shares, and stores this information in new columns in the ***stock_data*** DataFrame. 
- The ***fillna(0)*** function is used to set the first column of shares to be traded to 0, since there is no previous value to compare it to and avoid ***NaN***.

Finally, the code calculates the total portfolio value by summing up the value of each asset in ***stock_data['Portfolio']***, the portfolio returns as we did with the individual stocks and visualizes the trading routines by showing the number of shares to trade for each asset in the first few rows of the stock_data DataFrame.

In [ ]:
# Calculate the value and shares for each asset in the portfolio
for asset in stocks:
    stock_data[f'{asset}_value'] = stock_data[asset] * shares_df[f'{asset}_shares']
    stock_data[f'{asset}_shares'] = shares_df[f'{asset}_shares']
    stock_data[f'Trade_{asset}_shares'] = np.floor(stock_data[f'{asset}_shares']) - np.floor(stock_data[f'{asset}_shares'].shift(1))

stock_data = stock_data.fillna(0) # Set 0 first column of shares to trade

# Calculate the total portfolio value
stock_data['Portfolio'] = stock_data[[f'{asset}_value' for asset in stocks]].sum(axis=1)
stock_data['Portfolio_ret'] = np.log(stock_data['Portfolio']/stock_data['Portfolio'].shift(1).dropna())

# Visualize Trading Routines, every new input will be our trading routine to keep the portfolio allocation
stock_data[[f'Trade_{asset}_shares' for asset in stocks]].head()

In [ ]:
# Let's visualize our assets and portfolio.
stock_data[[f'{asset}_ret' for asset in stocks]].cumsum().apply(np.exp).plot(figsize=(11, 8), alpha=0.5)
stock_data['Portfolio_ret'].cumsum().apply(np.exp).plot(label='Portfolio')
plt.legend(loc="upper left")

Here we will finish the first part of our project. We've covered the following steps:
1. Define a investment strategy. 25% permanent allocation on $n$ assets. Ideally would take us through different economic environments successfully.
    1. For more examples have a look at ***Ray Dalio all wheather portfolio*** or ***Harry Browne permanent porfolio***, and play with some assets, distributions and rebalances.
2. Retreive, clean and process the data. Include relevant metrics such returns, which will help us later on
3. Define the ***rebalance routine*** and get the ***trading routines*** to mantain our allocation.
4. Visualize the performance. 



### 2. Analysis of investment strategy
<hr>

Since is not the main purpose of this code, we won't go in depth about the analytics of the investment strategy, deeper statistics, metrics or risk analysis itself. I will share few lines about how you can quickly visualize a general report. However always pay attention to:
- Correlation between your strategy and assets.
- Correlation between strategies on your system.
- Risk Management

And more important, does the strategy, exposure and rebalance practices meet your initial criteria?

As said before, here you have a quick way to observe the performance. There are many ways to do it an you can go in depth as much as you like. Here we will use a simple function from ***pyfolio*** library for a quick report, since we've got our DataFrame prepared with the returns already calculated.

In [ ]:
import pyfolio.tears as py

tear_sheet= py.create_full_tear_sheet(stock_data['Portfolio_ret'], benchmark_rets=stock_data['SPY_ret'])

This allows to visually observe the strategy and get some insights from the charts and tables. From a quick overview we would see:

- Our strategy didn't do particularly well from the years 2008 - 2018. 
    - What could have caused this?
    - Is the distribution ok for this policy?
- However, performs better on a Hawkish Federal Reserve period.
    - What makes it work better?
    - Is temporary or sustained?
- It reduces volatility compared to benchmark.
    - What if the volatility is higher and returns lower?
    - Is that strategy aiming the right client?
- Etc, etc.

Then you can go deeper on more quantitative analysis, risks, etc. Potentially, a longer period rebalance, such monthly, quarterly or even yearly would help us to capture returns. As well modifications on the redistribution or assets would bring different results.

### 3. Convert our investment strategy into a ETF.
<hr>

Once we've decided to continue with the strategy, we will convert it into a ETF example. The main key of an ETF is to provide the investor exposure to a basket of assets what you can easily sell or buy on the market, potentially lowering his risk and helping on diversification. In order to do this, it will be few steps to follow once the strategy and the portfolio has been created.

- Define a initial number of shares what the investor would be able to buy-sell.
- Since we don't have market data for our ETF share, we will need to simulate one.
- We will need to implement a system which controls the ratio between the market ETF price and the portfolio. We will do this by controling the number of oustanding shares.

The code sets up a new column in the stock_data DataFrame called sim_ETF_price and initializes its first value.

The initial value of the ETF will be calculated based on the initial portfolio value and the number of initial ETF shares.

In [ ]:
# Set the initial price. Portfolio value / shares
stock_data['sim_ETF_price'] = pd.Series(data=np.nan, index=stock_data.index)
stock_data['sim_ETF_price'][0] = stock_data['Portfolio'][0] / initial_etf_shares


- Then, it applies a random noise to each value of the ETF price by adding or subtracting a random number between -0.05 and 0.05 times the current value of the ETF price. 
- The ***np.random.uniform*** function generates uniformly distributed random numbers within the given range. This simulates the volatility and unpredictability of the stock market. 
- The resulting ***sim_ETF_price*** values are not the real market prices, but rather a simulation of the ***possible*** market prices which allows us to implement strategies in order to balance the shares oustanding.

In [ ]:
# Price Generation
noise_scale = 0.05
stock_data['sim_ETF_price'] = (stock_data['Portfolio'] / initial_etf_shares) * (1 + np.random.uniform(-noise_scale, noise_scale, size=len(stock_data)))

In [ ]:
stock_data['sim_ETF_price'].plot(label='Simulated ETF Market Price', figsize=(16, 10))
(stock_data['Portfolio'] / initial_etf_shares).plot(label='Portfolio / Initial shares')
plt.legend(loc="upper left")

This code initializes a new column in the stock_data dataframe named shares_ETF which is filled with NaN values. 

- It then sets the first value of shares_ETF to be the initial_etf_shares $= 100000$ value.
- After that, it calculates the number of shares to hold for the ETF in each subsequent time period. ***Dividing the Portfolio value by the simulated ETF price in each period*** 
- Finally, it rounds down the values in ***shares_ETF*** to the nearest integer using ***np.floor()***. 

The resulting ***shares_ETF*** column will indicate the number of shares to hold in the ETF at each time period in order to maintain the desired asset allocation and a fair value of the price based on the portfolio holding.

In [ ]:
stock_data['shares_ETF'] = pd.Series(data=np.nan, index=stock_data.index)
stock_data['shares_ETF'][0] = initial_etf_shares
stock_data['shares_ETF'][1:] = stock_data['Portfolio'][1:] / stock_data['sim_ETF_price'][1:]
stock_data['shares_ETF'] = np.floor(stock_data['shares_ETF'])

Now, similar to what we did to keep the balance of 25% for each asset on the previous step, we will keep track on how the ETF shares should change in order to keep the value in ***stock_data['Trade_ETF_shares'].*** 

For this we will do a report about how the number changes over time. It is worth nothing once we know the ***Portfolio Value*** and the ***Number of ETF shares***, we can get the ***Net Asset Value*** by:

$NAV = PortfolioValue / SharesETF$

This will allow us to track if our system is working effectively by monitoring the ETF market price and the NAV. 

As well we could get useful metrics such ETF Market Cap and compare it with our Portfolio size. Both metrics should be similar.

In [ ]:
stock_data['Trade_ETF_shares'] = np.floor(stock_data['shares_ETF']) - np.floor(stock_data['shares_ETF'].shift(1))
stock_data= stock_data.fillna(0)

In [ ]:
stock_data['NAV'] = stock_data['Portfolio'] / stock_data['shares_ETF']
stock_data['ETF_MarketCap'] = stock_data['sim_ETF_price'] * stock_data['shares_ETF']

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Plot Differences ETF_price - NAV')
ax1.hist(stock_data['sim_ETF_price'][1:] - stock_data['NAV'][1:])

Once we've checked if our system is workikng you can select small parts of your dataframe to perform different functions. 
For example:
- Get the trading orders to fill (as we will do on the following example)
- Access to returns for metrics
- Observe the holdings
- Etc, etc.

In [ ]:
holdings = stocks + ['ETF']
to_trade = stock_data[[f'Trade_{asset}_shares' for asset in holdings]]

In [ ]:
to_trade

In [ ]:
shares = stock_data[[f'{asset}_shares' for asset in stocks]]
shares.iloc[-1].plot.barh(figsize=(10,5), title='Porfolio Shares Holding')


In [ ]:
stock_data.columns

In [ ]:
stock_data['shares_ETF'].plot(figsize=(12,6), title='ETF Shares Change')
plt.axhline(initial_etf_shares, color= 'r', linestyle = '--')



Since our strategy was designed to rebalance the value daily, our portfolio keeps the same distribution (25%) per asset at every point.

In [ ]:
stock_data[[f'{asset}_value' for asset in stocks]].iloc[-1].plot.barh(figsize=(10,5), title='Porfolio Shares Holding')

In [ ]:
stock_data.head()